<a href="https://colab.research.google.com/github/MuhammedAshraf2020/imgCLF-using-Pytorch/blob/main/Image_Classification_using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader

In [3]:
# Import Data

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohamedashraf2020","key":"cd432666427ed4fad82b3c48c33bb008"}'}

In [5]:
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c dogs-vs-cats

100% 271M/271M [00:01<00:00, 186MB/s]
100% 271M/271M [00:01<00:00, 165MB/s]
 96% 523M/543M [00:03<00:00, 184MB/s]
100% 543M/543M [00:03<00:00, 155MB/s]
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 89.0MB/s]


In [7]:
#UnZip Data

In [ ]:
!unzip /content/train.zip

In [ ]:
!unzip /content/test1.zip

In [10]:
# Annotate train - test Data

In [11]:
!python annotate.py --name train --save train

25000it [01:09, 359.98it/s]


In [12]:
!python annotate.py --name test1 --save test

12500it [00:30, 406.66it/s]


In [2]:
from CustomDataSet import CustomDataSet

In [3]:
train_dataset = CustomDataSet(csv_file = "train.csv" , root_dir = "train" , transform = transforms.ToTensor())
test_dataset = CustomDataSet(csv_file = "test.csv" , root_dir = "train" , transform = transforms.ToTensor())

In [4]:
train_loader = DataLoader(dataset = train_dataset , batch_size = 150 , shuffle = True)
test_loader  = DataLoader(dataset = test_dataset  , batch_size = 150 , shuffle = True)

In [5]:
#Set VGG19 model

In [6]:
from model import model_arch , VGG

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
vgg_model = VGG(model_arch , 3 , 224 , 2).to(device = device)

In [9]:
# Train Model

In [10]:
critreion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg_model.parameters() , lr = 1e-3)

In [11]:
def CalcAccuracy(generator , model):
  num_correct = 0  
  num_samples = 0
  with torch.no_grad():
    for x , y in generator:
      x = x.float().to(device = "cuda")
      y = y.float().to(device = "cuda")
      scores = model(x)
      _ , prediction = scores.max(1)
      num_correct += (prediction == y).sum()
      num_samples += prediction.size(0)
    acc = (num_correct / num_samples) * 100
  return acc

In [ ]:
for epoch in range(5):
  for (data , target) in train_loader:
    data    = data.float().to(device = device)
    target  = target.to(device = device)
    scores  = vgg_model(data)
    loss    = critreion(scores , target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  train_acc   = CalcAccuracy(train_loader , vgg_model)
  test_acc   = CalcAccuracy(test_loader  , vgg_model)
  print("epoch {} train_acc = {:.3f} , test_acc = {:.3f}".format(epoch , train_acc , test_acc))

In [ ]:
for i , j , k , m in train_loader:
  print(i , j , k , m)